In [ ]:
import numpy as np
from music21 import *

%run lstm_model.ipynb
def generator_X(notes):
    t = 0
    rand_time = random.randint(0, notes.shape[0] - 16)
    while True:
        X = []
#         if t < notes.shape[0] - 16:
#             X.append(notes[t:t+16])
#         else: 
#             t = 0
#             continue
#         t += 16
        if rand_time < notes.shape[0] - 17:
            X.append(notes[rand_time:rand_time+16])
        else:
            rand_time = random.randint(0, notes.shape[0] - 16)
            X.append(notes[rand_time:rand_time+16])
        rand_time += 16
        X = np.array(X)
        yield (X)
        
def generate_notes(notes):
    output_indices = []
    for i in range(len(parts_index)):
        model = load_model("models/bach"+str(i)+".h5")
#     model = load_model(lstm_model.name+".h5")
        gen = ((X) for (X) in generator_X(notes))
#     predict_list = model.predict(X, batch_size=32)
        predict_list = model.predict_generator(gen, steps=500)
        part_notes = []
        for time in range(predict_list.shape[0]):
            for note_index in range(predict_list.shape[1]):
                part_notes.append(np.argmax(predict_list[time][note_index]))
        output_indices.append(part_notes)
    output_indices = np.array(output_indices)
    output_chorale = index_note_transform(output_indices, "to_note", len(parts_index))
    #output_indices = np.transpose(output_indices)
   # output_chorale = np.transpose(output_chorale)
    midi = chorale_to_midi(output_chorale)
    print(output_chorale)
    return midi
    
# def notes_to_midi(n, d, new_note, part):
#     if (len(n) < 2): # note or rest
#         if n[0] == 132:
#             part.append(note.Rest())
#         elif n[0] < 128:
#             if d > 0:
#                 new_note.duration = duration.Duration(d / 4)
#                 part.append(new_note)
#                 d = 1
#                 new_note = note.Note(n[0])
#             else:
#                 d += 1
#     else: # chord
#         if d > 0:
#             new_note.duration = duration.Duration(d / 4)
#             part.append(new_note)
#             d = 1
#             new_note = chord.Chord(n)
#         else:
#             d += 1
def chorale_to_midi(chorale):
    score = stream.Score()
    for i, chorale_part in enumerate(chorale):
        part = stream.Part(id=i)
        d = 0
        new_note = note.Rest()
        for sound in chorale_part:
            sound = [n for n in sound]
            if (len(sound) < 2): # note or rest
                if sound[0] == 132:
                    part.append(note.Rest())
                elif sound[0] < 128:
                    if d > 0:
                        new_note.duration = duration.Duration(d / 4)
                        part.append(new_note)
                    d = 1
                    new_note = note.Note(sound[0])
                else:
                    d += 1
            else: # chord
                if d > 0:
                    new_note.duration = duration.Duration(d / 4)
                    part.append(new_note)
                d = 1
                new_note = chord.Chord(sound)
            
        new_note.duration = duration.Duration(d / 4)
        part.append(new_note)
        score.insert(part)
    return score